# Accessing the Dataset

I manually downloaded the dataset since it was sufficiently small enough. You can download and access it here: https://www.kaggle.com/competitions/march-machine-learning-mania-2023


# Downloading the Dataset into the Local Environment

In [ ]:

import numpy as np
import pandas as pd
import os, re, copy
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

dataPath = "/content/drive/MyDrive/march-machine-learning-mania-2023"
csvFiles = dict()
for filename in os.listdir(dataPath):
  name = filename[0:len(filename)-4]
  if (not (name[0] == 'M' or name == "Cities" or name == "Conferences" or name == "SampleSubmission2023")):
    continue
  filePathTmp = dataPath + "/" + filename
  if (name == "MTeamSpellings"):
    csvFiles[name] = pd.read_csv(filePathTmp, encoding = "mac_roman")
  else:
    csvFiles[name] = pd.read_csv(filePathTmp)

Mounted at /content/drive


In [ ]:
tourney_results = csvFiles["MNCAATourneyDetailedResults"].copy()
regular_results = csvFiles['MRegularSeasonDetailedResults'].copy()
seeds = csvFiles['MNCAATourneySeeds'].copy()
df_test = csvFiles["SampleSubmission2023"].copy()
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIDA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIDB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))
NCAATournamentTeams = np.unique(np.concatenate([np.unique(df_test['TeamIDA']),np.unique(df_test['TeamIDB'])]))
conferences = csvFiles["MTeamConferences"].copy()

In [ ]:
def preprocess_csv(df):
    df['WFGRate'] = df['WFGM'] / df['WFGA']
    df["WFG3Rate"] = df['WFGM3'] / df['WFGA3']
    df['WFGM2'] = df['WFGM'] - df['WFGM3']
    df['WFGA2'] = df['WFGA'] - df['WFGA3']
    df["WFG2Rate"] = df['WFGM2'] / df['WFGA2']

    df['LFGRate'] = df['LFGM'] / df['LFGA']
    df["LFG3Rate"] = df['LFGM3'] / df['LFGA3']
    df['LFGM2'] = df['LFGM'] - df['LFGM3']
    df['LFGA2'] = df['LFGA'] - df['LFGA3']
    df["LFG2Rate"] = df['LFGM2'] / df['LFGA2']

    # EFFG:
    df["WEFFG"] = (df['WFGM'] + 0.5 * df['WFGM3'])/df['WFGA']
    df["LEFFG"] = (df['LFGM'] + 0.5 * df['LFGM3'])/df['LFGA']


    # Number of Possessions (Estimated hyperparameter: .475)
    df['WPossessions'] = (df['WFGA'] - df['WOR']) + df['WTO'] + .475 * df['WFTA']
    df['LPossessions'] = (df['LFGA'] - df['LOR']) + df['LTO'] + .475 * df['LFTA']

    # TODO (2023): Adjusted PtsPerPoss/PtsAllowedPerPoss?
    # Points Per Possession
    df['WPtsPerPoss'] = df['WScore'] / df['WPossessions']
    df['LPtsPerPoss'] = df['LScore'] / df['LPossessions']

    # # Points Allowed Per Possession:
    # df['WPtsAllowedPerPoss'] = df['LScore'] / df['LPossessions']
    # df['LPtsAllowedPerPoss'] = df['WScore'] / df['WPossessions']

    # Percentage of Field Goals Assisted
    df['WAssistRate'] = df['WAst'] / df['WFGM']
    df['LAssistRate'] = df['LAst'] / df['LFGM']

    # Offensive Rebound Percentage
    df['WOReboundPct'] = df['WOR'] / (df['WFGA'] - df['WFGM'])
    df['LOReboundPct'] = df['LOR'] / (df['LFGA'] - df['LFGM'])

    # Defensive Rebound Percentage
    df['WDReboundPct'] = df['WDR'] / (df['LFGA'] - df['LFGM'])
    df['LDReboundPct'] = df['LDR'] / (df['WFGA'] - df['WFGM'])

    # Assist to Turnover Ratio
    df['WATORatio'] = df['WAst'] / df['WTO']
    df['LATORatio'] = df['LAst'] / df['LTO']

    # Turnover Rate
    df['WTORate'] = df['WTO'] / df['WPossessions']
    df['LTORate'] = df['LTO'] /  df['LPossessions']

    # Free Throw Rate
    df['WFTRate'] = df['WFTA'] / df['WFGA']
    df['LFTRate'] = df['LFTA'] /  df['LFGA']

    # Block Pct:

    df['WBlockRate'] = df['WBlk'] / df['LFGM2']
    df['LBlockRate'] = df['LBlk'] / df['WFGM2']

    df = df.rename(columns={"WLoc":"location"})
    
    # TODO: calculate offensive/defensive efficiency & adj. off/def efficiency
    # TODO: Conference Tournament Record
    # TODO: KenPom Ranking
    # TODO: Spread

    # TODO: For Summary Statistics: Seed, Conference Record, 4-yr and 20-yr team record, Coach Record
    # TODO: Number of Injuries currently. If possible, number of starting injuries


    # # Check if removing the matches not containing teams in the tournament would result in teams
    # # not being in the df:
    # amountOfTeamsNotInFilteringDFW = len(list(set(NCAA2022Teams) - set(df["WTeamID"])))
    # amountOfTeamsNotInFilteringDFL = len(list(set(NCAA2022Teams) - set(df["LTeamID"])))
    # if (amountOfTeamsNotInFilteringDFL == 0 and amountOfTeamsNotInFilteringDFW == 0):
    # df = df.loc[(df['WTeamID'].isin(NCAATournamentTeams)) & (df['LTeamID'].isin(NCAATournamentTeams))]

    return df

regular_results = preprocess_csv(csvFiles['MRegularSeasonDetailedResults'].copy())
regular_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,location,NumOT,WFGM,WFGA,...,WDReboundPct,LDReboundPct,WATORatio,LATORatio,WTORate,LTORate,WFTRate,LFTRate,WBlockRate,LBlockRate
0,2003,10,1104,68,1328,62,N,0,27,58,...,0.774194,0.709677,0.565217,0.444444,0.304434,0.251924,0.310345,0.415094,0.050000,0.083333
1,2003,10,1272,70,1393,63,N,0,26,62,...,0.651163,0.694444,1.230769,0.583333,0.188338,0.175182,0.306452,0.298507,0.222222,0.333333
2,2003,11,1266,73,1437,61,N,0,24,58,...,0.509804,0.647059,1.500000,0.750000,0.154381,0.184829,0.500000,0.315068,0.105263,0.312500
3,2003,11,1296,56,1457,50,N,0,18,38,...,0.612903,1.000000,0.916667,0.473684,0.204342,0.326882,0.815789,0.306122,0.166667,0.200000
4,2003,11,1400,77,1208,71,N,0,30,61,...,0.578947,0.483871,0.857143,1.200000,0.218153,0.156678,0.213115,0.435484,0.222222,0.041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107629,2023,132,1104,82,1401,63,N,0,27,66,...,0.644444,0.512821,1.333333,0.461538,0.177910,0.182073,0.333333,0.375000,0.500000,0.133333
107630,2023,132,1272,75,1222,65,A,0,27,53,...,0.613636,0.769231,1.000000,1.100000,0.169296,0.153139,0.396226,0.437500,0.142857,0.190476
107631,2023,132,1343,74,1463,65,H,0,23,57,...,0.892857,0.735294,2.200000,1.200000,0.080128,0.158667,0.421053,0.380000,0.142857,0.500000
107632,2023,132,1345,67,1336,65,N,0,24,62,...,0.862069,0.710526,2.428571,1.400000,0.106870,0.155945,0.322581,0.294118,0.000000,0.111111


In [ ]:
tmp2 = list()
tmp2.append("9")
print(tmp2)

['9']


In [ ]:
def determine_streak(df, win = True):
  if (win): streak_label = "WTeamID"; df_label = "win_streak"
  else: streak_label = "LTeamID"; df_label = "lose_streak"
  num_seasons = sorted(list(set(df["Season"])))
  season_ls = list()
  team_id_ls = list()
  streak_ls = list()
  for season in num_seasons:
    df_for_season = df.loc[(df["Season"] == season), :]
    # obtain all the teams in the respective season:
    team_ids = sorted(list(set(list(set(df_for_season["WTeamID"])) + list(set(df_for_season["LTeamID"])))))
    for team_id in team_ids:
      df_with_season_and_team = df_for_season.loc[((df_for_season["WTeamID"] == team_id) | (df_for_season["LTeamID"] == team_id)), :]
      n_games = df_with_season_and_team.shape[0]
      streak = 0
      for i in reversed(range(n_games)):
        if (df_with_season_and_team.iloc[i][streak_label] == team_id): streak += 1
        else: break
      season_ls.append(season)
      team_id_ls.append(team_id)
      streak_ls.append(streak)

      
  result_df = pd.DataFrame(zip(season_ls, team_id_ls, streak_ls), columns =['Season', 'TeamID', df_label])
  return(result_df)
  

win_streaks = determine_streak(regular_results.copy())
lose_streaks = determine_streak(regular_results.copy(), win = False)

In [ ]:
def prepare_data(df):
    swapcols = [x.replace('W','$').replace('L','W') for x in list(df.columns)]
    swapcols = [x.replace('$','L') for x in swapcols]
    dfswap = df[swapcols]

    dfswap.loc[df['location'] == 'H', 'location'] = 'A'
    dfswap.loc[df['location'] == 'A', 'location'] = 'H'
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap])
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)

    output['PointDiff'] = output['T1_Score'] - output['T2_Score']

    return output
regular_data = prepare_data(regular_results.copy())
regular_data

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_DReboundPct,T1_ATORatio,T2_ATORatio,T1_TORate,T2_TORate,T1_FTRate,T2_FTRate,T1_BlockRate,T2_BlockRate,PointDiff
0,2003,10,1104,68,1328,62,0,0,27,58,...,0.709677,0.565217,0.444444,0.304434,0.251924,0.310345,0.415094,0.050000,0.083333,6
1,2003,10,1272,70,1393,63,0,0,26,62,...,0.694444,1.230769,0.583333,0.188338,0.175182,0.306452,0.298507,0.222222,0.333333,7
2,2003,11,1266,73,1437,61,0,0,24,58,...,0.647059,1.500000,0.750000,0.154381,0.184829,0.500000,0.315068,0.105263,0.312500,12
3,2003,11,1296,56,1457,50,0,0,18,38,...,1.000000,0.916667,0.473684,0.204342,0.326882,0.815789,0.306122,0.166667,0.200000,6
4,2003,11,1400,77,1208,71,0,0,30,61,...,0.483871,0.857143,1.200000,0.218153,0.156678,0.213115,0.435484,0.222222,0.041667,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107629,2023,132,1401,63,1104,82,0,0,19,64,...,0.644444,0.461538,1.333333,0.182073,0.177910,0.375000,0.333333,0.133333,0.500000,-19
107630,2023,132,1222,65,1272,75,1,0,20,64,...,0.613636,1.100000,1.000000,0.153139,0.169296,0.437500,0.396226,0.190476,0.142857,-10
107631,2023,132,1463,65,1343,74,-1,0,22,50,...,0.892857,1.200000,2.200000,0.158667,0.080128,0.380000,0.421053,0.500000,0.142857,-9
107632,2023,132,1336,65,1345,67,0,0,22,51,...,0.862069,1.400000,2.428571,0.155945,0.106870,0.294118,0.322581,0.111111,0.000000,-2


In [ ]:
summary_statistics = regular_data.groupby(["T1_TeamID", "Season"]).mean().drop(["location", "NumOT", "DayNum", "PointDiff", "T2_TeamID"], axis = 1).reset_index().rename(columns = {"T1_TeamID": "TeamID"})
summary_statistics.columns = summary_statistics.columns.str.replace("T2_", "T1_Opp_")
summary_statistics

summary_statistics_t2 = summary_statistics.copy()
summary_statistics_t2.columns = summary_statistics_t2.columns.str.replace("T1_", "T2_")
summary_statistics_t2.columns

Index(['TeamID', 'Season', 'T2_Score', 'T2_Opp_Score', 'T2_FGM', 'T2_FGA',
       'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast',
       'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 'T2_Opp_FGM', 'T2_Opp_FGA',
       'T2_Opp_FGM3', 'T2_Opp_FGA3', 'T2_Opp_FTM', 'T2_Opp_FTA', 'T2_Opp_OR',
       'T2_Opp_DR', 'T2_Opp_Ast', 'T2_Opp_TO', 'T2_Opp_Stl', 'T2_Opp_Blk',
       'T2_Opp_PF', 'T2_FGRate', 'T2_FG3Rate', 'T2_FGM2', 'T2_FGA2',
       'T2_FG2Rate', 'T2_Opp_FGRate', 'T2_Opp_FG3Rate', 'T2_Opp_FGM2',
       'T2_Opp_FGA2', 'T2_Opp_FG2Rate', 'T2_EFFG', 'T2_Opp_EFFG',
       'T2_Possessions', 'T2_Opp_Possessions', 'T2_PtsPerPoss',
       'T2_Opp_PtsPerPoss', 'T2_AssistRate', 'T2_Opp_AssistRate',
       'T2_OReboundPct', 'T2_Opp_OReboundPct', 'T2_DReboundPct',
       'T2_Opp_DReboundPct', 'T2_ATORatio', 'T2_Opp_ATORatio', 'T2_TORate',
       'T2_Opp_TORate', 'T2_FTRate', 'T2_Opp_FTRate', 'T2_BlockRate',
       'T2_Opp_BlockRate'],
      dtype='object')

In [ ]:
tourney_results = preprocess_csv(tourney_results)



In [ ]:
# merge win and lose streaks to tournament_results

tourney_results = pd.merge(
    tourney_results.copy(),
    win_streaks.copy(), 
    how='left', 
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'win_streak': 'Wwin_streak'})

tourney_results = pd.merge(
    tourney_results.copy(),
    win_streaks.copy(), 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'win_streak': 'Lwin_streak'})

In [ ]:
tourney_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,location,NumOT,WFGM,WFGA,...,WATORatio,LATORatio,WTORate,LTORate,WFTRate,LFTRate,WBlockRate,LBlockRate,Wwin_streak,Lwin_streak
0,2003,134,1421,92,1411,84,N,1,32,69,...,1.416667,1.066667,0.151229,0.188147,0.376812,0.462687,0.176471,0.000000,3,5
1,2003,136,1112,80,1436,51,N,0,31,66,...,1.375000,0.705882,0.206053,0.222732,0.212121,0.109375,0.437500,0.125000,0,4
2,2003,136,1113,84,1272,71,N,0,31,59,...,2.000000,0.916667,0.131483,0.169074,0.372881,0.304348,0.222222,0.200000,0,0
3,2003,136,1141,79,1166,73,N,0,29,53,...,0.833333,0.952381,0.250435,0.279720,0.471698,0.283333,0.050000,0.230769,6,4
4,2003,136,1143,76,1301,74,N,1,27,64,...,1.307692,1.142857,0.185913,0.201439,0.359375,0.357143,0.125000,0.400000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,2022,146,1242,76,1274,50,N,0,29,58,...,1.800000,0.500000,0.134499,0.199501,0.448276,0.236364,0.187500,0.166667,5,0
1244,2022,146,1314,69,1389,49,N,0,25,61,...,2.500000,1.571429,0.092343,0.103321,0.344262,0.166667,0.357143,0.368421,0,7
1245,2022,152,1242,81,1437,65,N,0,29,54,...,2.571429,1.333333,0.122431,0.153191,0.240741,0.175439,0.444444,0.000000,5,5
1246,2022,152,1314,81,1181,77,N,0,27,64,...,0.700000,3.000000,0.142045,0.055172,0.375000,0.277778,0.200000,0.235294,0,0


In [ ]:
tourney_data = prepare_data(tourney_results.copy())
tourney_data

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,...,T2_ATORatio,T1_TORate,T2_TORate,T1_FTRate,T2_FTRate,T1_BlockRate,T2_BlockRate,T1_win_streak,T2_win_streak,PointDiff
0,2003,134,1421,92,1411,84,0,1,32,69,...,1.066667,0.151229,0.188147,0.376812,0.462687,0.176471,0.000000,3,5,8
1,2003,136,1112,80,1436,51,0,0,31,66,...,0.705882,0.206053,0.222732,0.212121,0.109375,0.437500,0.125000,0,4,29
2,2003,136,1113,84,1272,71,0,0,31,59,...,0.916667,0.131483,0.169074,0.372881,0.304348,0.222222,0.200000,0,0,13
3,2003,136,1141,79,1166,73,0,0,29,53,...,0.952381,0.250435,0.279720,0.471698,0.283333,0.050000,0.230769,6,4,6
4,2003,136,1143,76,1301,74,0,1,27,64,...,1.142857,0.185913,0.201439,0.359375,0.357143,0.125000,0.400000,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,2022,146,1274,50,1242,76,0,0,19,55,...,1.800000,0.199501,0.134499,0.236364,0.448276,0.166667,0.187500,0,5,-26
1244,2022,146,1389,49,1314,69,0,0,18,60,...,2.500000,0.103321,0.092343,0.166667,0.344262,0.368421,0.357143,7,0,-20
1245,2022,152,1437,65,1242,81,0,0,22,57,...,2.571429,0.153191,0.122431,0.175439,0.240741,0.000000,0.444444,5,5,-16
1246,2022,152,1181,77,1314,81,0,0,30,72,...,0.700000,0.055172,0.142045,0.277778,0.375000,0.235294,0.200000,0,0,-4


In [ ]:
tmp = tourney_data[["Season", "DayNum", "T1_TeamID", "T2_TeamID", "T1_win_streak", "T2_win_streak", "PointDiff"]]
tmp = pd.merge(
    tmp,
    summary_statistics.copy(), 
    how='left', 
    left_on=['Season', 'T1_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns = {"T1_Score": "T1_PPG", "T1_Opp_Score": "T1_Opp_PPG"})
tourney_data = pd.merge(
    tmp,
    summary_statistics_t2.copy(), 
    how='left', 
    left_on=['Season', 'T2_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns = {"T2_Score": "T2_PPG", "T2_Opp_Score": "T2_Opp_PPG"})
tourney_data

,Season,DayNum,T1_TeamID,T2_TeamID,T1_win_streak,T2_win_streak,PointDiff,T1_PPG,T1_Opp_PPG,T1_FGM,...,T2_DReboundPct,T2_Opp_DReboundPct,T2_ATORatio,T2_Opp_ATORatio,T2_TORate,T2_Opp_TORate,T2_FTRate,T2_Opp_FTRate,T2_BlockRate,T2_Opp_BlockRate
0,2003,134,1421,1411,3,5,8,71.206897,78.448276,24.379310,...,0.724640,0.758459,0.983780,1.095845,0.215143,0.199854,0.529491,0.318144,0.126256,0.148191
1,2003,136,1112,1436,0,4,29,85.214286,70.250000,30.321429,...,0.802653,0.699075,1.077472,1.160533,0.212863,0.194157,0.356274,0.289028,0.199335,0.211201
2,2003,136,1113,1272,0,0,13,75.965517,69.172414,27.206897,...,0.765180,0.712062,1.293030,0.949968,0.194125,0.213262,0.396624,0.370114,0.333534,0.177554
3,2003,136,1141,1166,6,4,6,79.344828,73.241379,26.620690,...,0.744973,0.746488,1.331533,0.791461,0.192564,0.244220,0.354147,0.305944,0.249581,0.136628
4,2003,136,1143,1301,0,0,2,74.482759,69.758621,27.344828,...,0.755463,0.745138,1.095062,0.999924,0.209940,0.218002,0.387496,0.409074,0.178108,0.207845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,2022,146,1274,1242,0,5,-26,74.818182,71.030303,27.575758,...,0.717460,0.686686,1.457294,1.121654,0.169606,0.169282,0.335013,0.282414,0.244540,0.154302
2492,2022,146,1389,1314,7,0,-20,66.172414,62.241379,23.206897,...,0.790277,0.697128,1.479278,1.485100,0.157818,0.135355,0.300486,0.216706,0.217992,0.171638
2493,2022,152,1437,1242,5,5,-16,72.636364,63.090909,24.606061,...,0.717460,0.686686,1.457294,1.121654,0.169606,0.169282,0.335013,0.282414,0.244540,0.154302
2494,2022,152,1181,1314,0,0,-4,80.176471,67.147059,29.676471,...,0.790277,0.697128,1.479278,1.485100,0.157818,0.135355,0.300486,0.216706,0.217992,0.171638


In [ ]:
def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

seeds["Seed"] = seeds["Seed"].apply(treat_seed)
seeds

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374
...,...,...,...
2485,2023,12,1433
2486,2023,13,1233
2487,2023,14,1213
2488,2023,15,1421


In [ ]:
# merge seed and conference data:
tournament_with_seeds_df = pd.merge(
    tourney_data.copy(),
    seeds.copy(), 
    how='left', 
    left_on=['Season', 'T1_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'T1_Seed'})

tournament_with_seeds_df = pd.merge(
    tournament_with_seeds_df.copy(),
    seeds.copy(), 
    how='left', 
    left_on=['Season', 'T2_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'T2_Seed'})

tournament_with_seeds_and_conferences_df = pd.merge(
    tournament_with_seeds_df.copy(),
    conferences.copy(), 
    how='left', 
    left_on=['Season', 'T1_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'ConfAbbrev': 'T1_Conference'})

tournament_with_seeds_and_conferences_df = pd.merge(
    tournament_with_seeds_and_conferences_df.copy(),
    conferences.copy(), 
    how='left', 
    left_on=['Season', 'T2_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'ConfAbbrev': 'T2_Conference'})

regular_with_conferences_df = pd.merge(
    regular_data.copy(),
    conferences.copy(), 
    how='left', 
    left_on=['Season', 'T1_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'ConfAbbrev': 'T1_Conference'})

regular_with_conferences_df = pd.merge(
    regular_with_conferences_df.copy(),
    conferences.copy(), 
    how='left', 
    left_on=['Season', 'T2_TeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'ConfAbbrev': 'T2_Conference'})

In [ ]:
returned_df = tournament_with_seeds_and_conferences_df

In [ ]:
team_spellings = csvFiles["MTeamSpellings"].copy()
team_spellings_first = team_spellings.drop_duplicates(subset=['TeamID'])
team_spellings_first[team_spellings_first["TeamID"] == 1113]

,TeamNameSpelling,TeamID
40,arizona st,1113


In [ ]:
returned_df = pd.merge(
    returned_df.copy(),
    team_spellings_first.copy(), 
    how='left', 
    left_on=['T1_TeamID'], 
    right_on=['TeamID']
).drop('TeamID', axis=1).rename(columns={'TeamNameSpelling': 'T1'})

returned_df = pd.merge(
    returned_df.copy(),
    team_spellings_first.copy(), 
    how='left', 
    left_on=['T2_TeamID'], 
    right_on=['TeamID']
).drop('TeamID', axis=1).rename(columns={'TeamNameSpelling': 'T2'})

In [ ]:
# Export dataset with streak and compiled features as a .csv file:

returned_df.to_csv("mensMarchMadnessDataAllRounds.csv")